# Code Clarity Tutorial - simple python code search example 
In this notebook we hope to go through a simple example of how to use codeclarity to generate vectors of source
code and natrual language to build a simple vector retrieval application to power a very simple search engine. 

## Through the course of this notebook, we will cover-
1. The basics of semantic search
2. The applications of dense vectors for information retrieval
3. A model that was finetuned for code search of python-english source code. 
4. Implimenting a simple search of an example large repository


## Goal : semantic search of code 

To quote from Neil Reimers project [sentence transformers](https://www.sbert.net/examples/applications/semantic-search/README.html), which is an excellent resource for learning about dense vectors and their uses in NLP at large:

"*Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines which only find documents based on lexical matches, semantic search can also find synonyms.*"


This means that semantic search differs from it's predecessor, lexical search, in that instead of looking specific patterns that match in a retrieval object such as keyword matching, we aim to use machine learning models to capture a deeper contextual meaning of objects that we can map to a textual description- namely a query.

 This is an idea that has been adopted to great success to power every major search engine in the past 5 years, and has the benefit of being able to power search for arbitary sequences of symbols, be them [text](https://www.sbert.net/docs/pretrained_models.html), [images](https://openai.com/blog/clip/) or [audio](https://arxiv.org/abs/1904.05073).
 


## Applications of dense vectors
### Sparse VS Dense Vectors
Before the common prevelence of deep learning models in natrual language processing across machine learning, before the use of  release of [BERT](https://arxiv.org/abs/1706.03762) and [Word2Vec Varients](https://arxiv.org/abs/1301.3781), a common mathmatical representation of textual features was [TF-IDF](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3): the Term Frequency Inverse Document Frequency of a sequence.

 We won't explain it here, but TF-IDF is an example of a sparse vector representation. There are as many elements in the array storing the representation as there are unique words in the training corpus. This being said, a given piece of text, especially if it has a small vocabulary, will consist of mostly zeros. 

 This contrasts with for example, a [S-BERT sentence embedding](https://www.sbert.net/examples/applications/computing-embeddings/README.html), where no matter the length of the input string, it will be mapped to a fixed length array, the indices of whom are mostly nonzero integers. 

### Using dense vectors for information retrieval
Although the details of training a dense retrieval model to generate embedddings is outside the scope of this tutorial, we will note that in the process of the finetuning of a neural retrieval model, we directly compare queries to candidate items in such a way that the embedding representation of each is mapped into a shared metric space. This vector space can be exploited to generate a similarity score between a given query, and all the items in a search index. 

A common candidate used for search similarity at the embedding time is [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity), the formula for which is below: 

ADD FORMULA HERE 

This will output a score of similarity between 1 and -1, where a higher score denotes a better fit of an item in the search index to a query. A simple search implimentation, like the one in this tutorial may simply calculate the cosine similarity (or inner product for vector embeddings that are not normalized values) And perform a sort of the items in the index to find the 'nearest neibours' of the query.

## Models used
### UnixCoder and Code Understanding transformers 
This section will be kept short in the name of brevvity and the depth of this subdomain, but in short the finetuning of neural models for a given task will often fall short of state of the art performance if the foundational model has significant drift in its pretraining data from the data on the downstream task. 

For example, a model pretrained on an english corpus, say Bert-Large-Uncased, will likely have decreased performance on a text classification task in spanish. In a similar sense source code and natrual language unsuprisingly have stark differences in vocabulary, lexical structure and syntax. Thus, we start with a foundational model that has been shown to have strong understanding of both english and multiple programming languages as a result of its pretraining schema and data - [UniXCoder](THIS NEEDS A LINK)

Due to being trained on both source code and text, it proves highly suitable for code search tasks, as shown by its strong performance on prominent code search benchmarks like [CodeSearchNet](THIS NEEDS A LINK). Despite the release of the pretrained models by the authors, the full inference pipeline can prove tricky. This is where codeclarity comes in! 

## Implimenting a simple search of an example large repository
### Data, Stack and General Workflow 
To illustrate the use of a codesearch model, we try to build a code understanding tool using an opensource python repository of reasonable size - the AWS python SDK. This dataset spans well over 1000 functions and would require quite some onboarding for contribution in terms of finding where specific functionality is implimented.

Thus, we will use codeclarity to impliment a very simple code search tool which:
1. Embeds all of the code snippits in the repository to dense vectors
2. Builds an index of these vectors that is searchable for it's nearest neibours
3. Uses these neibour indicies to retrieve the function snippit and it's url on github

Lets get started! We will be using the pretrained checkpoints for UnixCoder-base : a code understanding model that in addition to being able to model our queries, understands python, javascript, java, go and ruby.

In [1]:
#Install the master branch of codeclarity
!pip install git+https://github.com/DocumaticAI/CodeClarity

  Cloning https://github.com/DocumaticAI/CodeClarity to /tmp/pip-req-build-kvvi14lj
  Running command git clone --filter=blob:none --quiet https://github.com/DocumaticAI/CodeClarity /tmp/pip-req-build-kvvi14lj
  fatal: unable to access 'https://github.com/DocumaticAI/CodeClarity/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/DocumaticAI/CodeClarity /tmp/pip-req-build-kvvi14lj did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/DocumaticAI/CodeClarity /tmp/pip-req-build-kvvi14lj did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
#Imports 
import codeclarity
import pandas as pd 
#import faiss
import numpy as np
import pathlib

In [3]:
df = pd.read_csv("aws_python_sdk.csv").drop(columns=["Unnamed: 0"])
df.head(5)

,code,language,name,filepath
0,"def read(fname):\n """"""\n Args:\n ...",python,read,setup.py
1,"def read_version():\n return read(""VERSION""...",python,read_version,setup.py
2,"def read_requirements(filename):\n """"""Reads...",python,read_requirements,setup.py
3,"def setup(app):\n sys.stdout.write(""Generat...",python,setup,doc/conf.py
4,"def get_jumpstart_sdk_manifest():\n url = ""...",python,get_jumpstart_sdk_manifest,doc/doc_utils/jumpstart_doc_utils.py


In [4]:
from codeclarity import CodeEmbedder
embedding_model = CodeEmbedder(base_model= "microsoft/unixcoder-base")

Search retrieval model for allowed_languages ['java', 'ruby', 'python', 'php', 'javascript', 'go'] loaded correctly to device cuda in 1.9021415710449219 seconds


In [5]:
df['embeddings'] = df['code'].map(lambda x: embedding_model.encode(x, language= "python",))

32it [00:00, 32.37it/s]              
32it [00:00, 1118.34it/s]            
32it [00:00, 1536.71it/s]            
32it [00:00, 1767.63it/s]            
32it [00:00, 1630.16it/s]            
32it [00:00, 1634.87it/s]            
32it [00:00, 1804.83it/s]            
32it [00:00, 1738.68it/s]            
32it [00:00, 1848.83it/s]            
32it [00:00, 1345.46it/s]            
32it [00:00, 1838.95it/s]            
32it [00:00, 1839.61it/s]            
32it [00:00, 1440.03it/s]            
32it [00:00, 1751.02it/s]            
32it [00:00, 1747.88it/s]            
32it [00:00, 1793.40it/s]            
32it [00:00, 1734.03it/s]            
32it [00:00, 1694.78it/s]            
32it [00:00, 1747.35it/s]            
32it [00:00, 1287.66it/s]            
32it [00:00, 978.02it/s]             
32it [00:00, 1242.39it/s]            
32it [00:00, 1636.26it/s]            
32it [00:00, 1296.25it/s]            
32it [00:00, 1536.23it/s]            
32it [00:00, 1880.41it/s]            
32it [00:00,

KeyboardInterrupt: 

In [13]:
list1 = [
    
]

In [9]:
list2 = [-9.4705e-03, -2.9795e-04,  8.3804e-03,  3.2826e-02,  7.9433e-04,
        -2.0464e-02,  3.0852e-02,  9.9523e-03,  9.8546e-03,  2.6226e-03,
        -1.0893e-02,  1.6409e-02, -4.5563e-03, -1.4114e-02,  1.5096e-02,
         7.1318e-03,  3.4828e-02,  2.5799e-02, -3.4286e-02,  2.9453e-02,
        -1.2645e-02, -1.5536e-03, -5.2210e-03, -1.8222e-02, -1.7984e-02,
        -1.9821e-02,  6.5271e-02,  3.5810e-02, -5.3572e-02, -8.5271e-03,
        -5.5045e-03, -6.2877e-02, -6.5003e-03, -1.9424e-02, -2.8050e-02,
         4.1085e-02,  4.7852e-02,  5.6224e-02,  4.1600e-02, -7.0654e-03,
        -2.3200e-03,  3.5124e-02,  1.1927e-02,  2.5134e-02,  5.5791e-02,
         1.8984e-02,  1.4872e-02,  1.1159e-02, -2.9389e-02, -9.3733e-03,
         9.1126e-03, -1.6851e-02, -1.4882e-03, -2.7097e-02, -3.5906e-02,
        -2.9724e-02, -9.6659e-02, -3.1729e-02,  2.5229e-02,  2.9723e-02,
        -6.7727e-03,  1.8323e-02, -9.3468e-03, -4.5748e-02, -5.2863e-02,
        -5.3058e-02,  5.9401e-02, -3.4622e-04,  7.8057e-03,  5.5638e-02,
        -4.4357e-02,  8.9698e-03, -1.2619e-02, -2.5132e-02,  3.4503e-02,
         1.0116e-02, -3.1288e-02,  6.3194e-02,  9.7071e-03, -9.0302e-02,
        -1.3334e-02, -5.4792e-04,  2.8675e-02, -3.4635e-02, -2.0076e-02,
        -1.0806e-02, -3.0514e-03,  1.5090e-02, -2.0776e-02, -4.1020e-03,
        -1.7474e-02,  2.5333e-02, -4.3945e-02,  1.4269e-02, -5.1848e-03,
         3.6166e-02,  5.3572e-02, -5.5955e-02,  2.6023e-02,  2.0124e-02,
        -7.5964e-02, -1.9964e-02, -1.5138e-02, -6.9391e-03, -2.3661e-02,
         4.2990e-02,  5.1144e-02, -3.1334e-03,  3.2787e-02, -1.3274e-03,
        -5.1549e-02,  2.5463e-02, -1.4303e-02, -9.2848e-03, -4.3994e-02,
         1.3783e-02,  3.6015e-02, -4.4131e-02, -1.7754e-03, -3.6993e-02,
        -1.5251e-02,  3.7513e-02,  5.5703e-02,  1.1912e-02, -4.7061e-03,
         3.5272e-02,  1.5108e-02, -2.1007e-02, -1.5103e-02,  4.5265e-03,
         4.3430e-02,  2.5040e-02, -3.2785e-03,  3.0312e-02, -2.9404e-02,
         5.4511e-03,  6.4546e-02,  4.2867e-02, -4.0134e-02,  1.3932e-02,
        -1.3196e-02,  3.1594e-02,  3.3119e-02, -1.5998e-02,  1.6759e-02,
        -4.3418e-02, -3.5840e-02,  1.5354e-02,  2.9100e-02, -1.7388e-03,
        -5.2197e-02, -5.2179e-02, -1.5233e-02, -2.4493e-02,  3.0805e-02,
        -2.3567e-02, -1.5455e-02,  2.6444e-02,  3.2410e-02,  1.0478e-02,
        -2.0803e-02, -7.7435e-03, -4.1645e-02, -7.1961e-03,  8.8731e-02,
         3.3428e-02,  1.9903e-02, -4.7201e-02,  3.2093e-02, -9.7264e-03,
         3.9736e-02,  3.4469e-02,  5.7976e-02, -4.1585e-03,  4.0362e-02,
         3.1724e-02,  2.3813e-02,  1.7337e-02,  2.0706e-02, -7.3654e-02,
         3.9492e-02, -5.1860e-03, -6.7842e-02, -1.1503e-02,  1.5095e-02,
         2.9963e-02,  5.1364e-02, -4.5150e-02,  1.1278e-02, -5.0196e-02,
         2.4135e-02,  1.0755e-02, -8.1792e-03,  2.6872e-02,  2.3595e-02,
         2.2986e-02,  2.2785e-02,  6.2184e-02, -6.8987e-03, -1.6251e-02,
         1.8614e-02, -8.0747e-02,  4.2781e-03,  3.2317e-02, -3.8991e-02,
         1.7351e-02, -4.6676e-03, -8.8876e-03, -1.7468e-02,  4.6114e-03,
        -3.4291e-02, -1.5281e-02,  1.2969e-02, -4.9512e-03,  9.2606e-03,
        -1.5531e-02, -2.4362e-02, -9.8341e-03, -5.3930e-02, -4.8061e-03,
         2.9951e-02, -7.7489e-03,  1.1930e-02, -4.5477e-02, -4.8811e-02,
        -1.8896e-02,  4.0656e-02,  5.1959e-02,  1.5554e-03,  2.1522e-02,
         3.5156e-02,  3.6120e-02,  4.4128e-02,  2.8157e-02,  4.6799e-03,
        -4.3041e-02,  9.5927e-03,  4.7003e-02,  2.8229e-02,  2.4599e-02,
        -3.1964e-02,  4.2688e-02,  2.0601e-02,  9.7161e-02, -2.8592e-02,
        -1.7215e-02, -1.1786e-02,  1.7368e-02,  5.2823e-02, -4.0432e-02,
        -1.8023e-02,  1.7358e-02, -6.6637e-03,  3.8529e-02, -2.6821e-02,
         5.1398e-03,  1.1095e-02, -7.3492e-03, -2.2771e-02, -2.7037e-02,
        -2.9795e-03, -4.0646e-03, -2.8492e-02,  1.8656e-02, -6.5753e-02,
        -2.4624e-02,  8.1590e-02, -8.4953e-03,  3.2194e-02,  2.5207e-02,
        -1.8299e-02, -1.8117e-03,  2.1915e-03,  3.8641e-02,  8.0773e-02,
         3.4106e-05,  1.7050e-01, -3.5325e-02,  2.9996e-02, -1.5018e-02,
         5.9896e-03,  1.5351e-02, -2.5429e-02, -5.1195e-03,  1.4382e-02,
        -6.7403e-03, -1.8937e-02, -3.5944e-02, -5.3868e-03,  6.4399e-03,
        -3.5354e-02, -1.4994e-02,  4.6496e-03,  3.4119e-02,  4.8816e-02,
        -4.3856e-03,  9.6534e-03, -2.3733e-02,  6.3321e-03,  4.0859e-02,
        -1.7091e-02, -9.8148e-03, -1.7974e-02,  2.2988e-02, -4.2939e-02,
        -1.1423e-02,  3.3794e-02, -6.9287e-03, -1.7658e-02,  3.1816e-02,
         2.2944e-02, -3.4311e-02,  4.5663e-03,  4.1951e-02, -4.2136e-02,
         1.7164e-02, -2.3376e-02, -2.8015e-02,  1.2567e-02,  1.8211e-03,
         7.1086e-03,  1.9287e-02, -3.1094e-02, -6.8010e-03,  4.3154e-03,
        -2.4207e-02,  5.1400e-02, -2.4324e-02,  2.9710e-02, -4.8201e-02,
         1.2596e-02,  2.4801e-02, -5.4003e-02, -7.6028e-04, -2.0052e-02,
        -5.4510e-03, -1.9132e-02,  1.3731e-02,  2.2952e-05, -8.8919e-03,
         2.5781e-02,  2.5278e-02,  4.6984e-02,  4.7637e-02,  9.9510e-03,
        -4.5115e-03,  1.0853e-02,  4.1468e-03, -4.8164e-02,  1.4033e-02,
         2.9132e-02, -5.9716e-03,  1.1723e-02,  7.4158e-03,  3.7190e-02,
         2.0371e-02,  1.3466e-02,  2.4084e-02,  1.1718e-02,  2.9856e-02,
        -2.0962e-02,  3.4836e-03, -3.9890e-02,  4.9323e-02,  7.0920e-02,
         2.0691e-02,  8.1536e-02, -1.4272e-04,  8.7795e-03,  1.1563e-02,
         4.8836e-03, -1.1261e-02, -1.1345e-02, -6.1483e-03, -7.0295e-03,
         2.1033e-02, -1.0946e-02, -1.1153e-02, -1.7776e-02, -1.1133e-02,
        -5.5678e-02, -3.1773e-02,  5.5101e-02, -3.1432e-02,  2.0926e-03,
        -1.4817e-02,  1.3377e-01, -1.0374e-02, -1.6471e-02,  3.6430e-02,
         2.3074e-02,  4.0526e-02, -1.6679e-02, -3.2976e-02,  1.8221e-02,
        -6.1285e-02,  6.8386e-02,  7.0551e-02,  1.9445e-02,  4.2229e-02,
         1.1880e-02,  8.2278e-03, -8.0128e-03, -6.4611e-02, -5.9548e-02,
         2.5779e-02, -2.3088e-02,  3.7977e-02,  1.8436e-02,  3.5293e-02,
         1.9051e-03,  3.9301e-02, -2.2641e-02, -9.5968e-03,  1.5926e-02,
        -2.2979e-03, -1.5087e-02, -3.0491e-02, -3.2144e-02, -1.1842e-02,
         5.9498e-04,  1.0807e-02, -1.0557e-02, -6.4826e-02, -5.9248e-02,
         2.4952e-02,  1.6776e-02,  8.3429e-03, -2.1735e-02, -3.6255e-02,
         2.8391e-02, -1.0284e-01, -2.3476e-02,  1.7145e-02,  2.0853e-02,
         1.3946e-02,  2.5844e-02,  1.9282e-02, -2.1943e-02, -1.3551e-02,
        -2.2878e-02, -1.8248e-02,  3.5017e-03, -9.6178e-03,  1.3824e-02,
        -3.2805e-03, -5.2594e-03, -2.3733e-02,  3.7061e-02,  2.8800e-02,
         3.9273e-02,  3.8089e-03, -5.0733e-02,  1.0110e-02, -9.8540e-03,
        -9.5206e-03,  2.5299e-02,  1.5812e-02, -1.0884e-02,  9.4506e-04,
         1.3242e-02, -3.2951e-02, -9.0614e-03,  2.5268e-02,  2.9481e-03,
         8.1084e-03,  2.5779e-02, -5.4949e-03,  1.1814e-02,  4.6325e-03,
        -4.9482e-03,  1.2539e-02, -1.2011e-02,  2.7745e-02, -8.0918e-02,
         2.1862e-02,  6.0894e-03,  3.4279e-02, -3.2131e-02,  1.5680e-02,
        -5.1510e-02,  4.7819e-03,  1.6720e-01, -2.0706e-05, -1.8546e-02,
         6.0092e-02,  5.5592e-03, -3.9077e-03,  4.2037e-02,  2.6204e-02,
         3.3520e-02, -2.1100e-03,  1.3935e-02, -9.8636e-03,  3.9217e-02,
        -1.0074e-02, -5.4186e-02, -1.7714e-02,  2.5253e-02, -5.7632e-02,
        -4.5139e-04,  5.8613e-02, -4.2356e-03,  2.0792e-02, -1.6430e-02,
        -6.7737e-03,  1.0145e-02,  1.4682e-02,  3.3827e-02, -1.8276e-02,
        -7.5568e-02, -7.3634e-02,  2.7692e-02, -5.1991e-02,  2.5967e-02,
        -4.7701e-02, -2.4168e-02,  5.3208e-03, -1.1329e-02,  1.1351e-02,
        -1.3428e-02, -2.4047e-02,  3.4317e-03, -4.0627e-02, -2.3399e-03,
         4.4249e-02, -9.0845e-02,  4.5868e-02,  6.0436e-02,  5.9474e-02,
         3.3197e-01,  2.2403e-02, -8.9355e-02, -1.9188e-02,  6.9659e-03,
        -2.7184e-02,  4.1527e-04,  4.9701e-02,  5.7460e-02,  1.7023e-02,
         1.7159e-02, -1.0416e-02,  1.2846e-03,  7.9628e-03, -1.3039e-02,
         1.1107e-02,  1.6863e-02, -1.8185e-02,  5.9576e-03, -2.6860e-02,
         2.1569e-02, -1.2359e-03, -1.6369e-02, -3.6709e-02, -2.5372e-02,
         4.4262e-02, -6.6998e-02, -1.4406e-02,  5.9738e-02, -9.1263e-03,
        -2.7707e-02, -1.8406e-02, -2.9887e-02,  2.6394e-02,  7.8930e-02,
        -1.6811e-02, -1.2083e-02, -2.2267e-02, -2.9172e-02,  1.6008e-03,
         8.0211e-03, -1.4703e-02,  2.3034e-02, -4.1874e-03, -6.2066e-03,
        -1.0202e-02, -2.4709e-03,  1.6578e-02, -1.7860e-02, -1.2980e-02,
        -1.0962e-02,  4.6960e-03, -3.6616e-02, -3.7174e-02,  9.3558e-03,
         5.5321e-02, -3.3334e-02, -5.7680e-02, -7.4629e-02,  1.3514e-02,
        -6.6639e-02, -7.5404e-03,  2.3242e-02,  5.8618e-03,  1.0760e-02,
         1.5024e-02,  2.7877e-02,  3.4445e-03,  1.2288e-02, -4.1133e-02,
        -1.9247e-02,  4.7947e-02,  1.8961e-02,  2.1627e-02, -2.1283e-02,
         7.0283e-03, -2.4404e-02, -2.3545e-02,  3.6536e-03,  1.7543e-02,
        -4.8873e-03,  1.6158e-01,  4.7321e-02, -1.4579e-01, -5.7729e-02,
        -1.3697e-02, -1.0912e-02,  2.3484e-02, -1.8035e-02, -3.0592e-02,
         2.4028e-02,  2.0970e-02, -4.4076e-02, -2.7445e-02,  8.7791e-02,
        -2.5390e-02, -2.7586e-02, -5.6429e-02,  1.6730e-03,  7.7917e-03,
        -1.6761e-02, -1.6862e-02, -4.1819e-02,  3.2826e-02,  2.7382e-02,
        -6.8793e-03,  1.5211e-02, -2.7734e-02,  2.4128e-02, -2.1624e-02,
         1.5267e-02,  3.0349e-02, -2.7122e-02,  2.4764e-02,  4.7492e-02,
         2.0906e-02,  6.0849e-03,  4.3399e-02, -2.7402e-03, -6.1110e-03,
         1.7974e-02,  4.7653e-02,  1.2890e-01,  2.7022e-02, -5.0018e-02,
         5.2912e-04, -6.9573e-02,  1.5808e-02,  4.2046e-02, -4.9990e-02,
         1.9867e-02, -3.7386e-02,  5.8342e-02,  4.8546e-02,  3.2969e-02,
        -4.1408e-02, -2.0893e-02, -1.2277e-02, -1.1339e-02, -4.1663e-02,
         1.4828e-02, -5.0326e-02, -2.1901e-02,  1.6179e-03,  2.3631e-02,
         5.3859e-02,  3.5624e-02,  7.1883e-03,  3.5379e-02, -3.3116e-02,
        -2.2658e-02, -2.7154e-02, -8.3837e-03, -2.9227e-02,  1.8018e-02,
        -1.2148e-03,  3.5684e-02, -2.9321e-02,  3.4548e-02,  3.6255e-02,
         4.5491e-02, -2.0940e-02,  1.2828e-02,  6.1681e-02,  2.8015e-02,
         5.7417e-02,  5.9698e-04,  3.5326e-02, -5.5446e-02, -7.4303e-03,
        -1.3638e-02, -6.6681e-03, -7.8643e-03,  1.4025e-03,  5.5879e-03,
        -4.5196e-03, -1.3845e-02, -3.3446e-03, -3.3333e-02, -3.0222e-02,
        -5.3758e-02, -1.5931e-02,  3.4496e-02, -3.1495e-02, -3.0046e-02,
        -1.4038e-02,  3.2426e-02, -6.1676e-02, -6.2093e-03,  1.2386e-02,
         1.6321e-02, -3.4065e-02, -1.3586e-02, -1.0316e-02,  2.0954e-03,
         3.2001e-02, -2.9873e-02, -3.6636e-02, -7.9641e-03, -2.4298e-02,
        -3.1017e-02, -2.0559e-02, -3.8405e-02, -8.0975e-03, -1.1743e-02,
         3.1967e-02,  4.9786e-03, -4.1152e-02,  3.4139e-02, -1.9716e-02,
        -3.7510e-02,  3.1266e-02,  2.2271e-02, -8.2954e-03,  4.6990e-02,
        -2.2757e-02,  8.9280e-03, -2.8307e-03,  7.3091e-04,  1.7553e-02,
        -9.8950e-02, -3.5606e-02,  4.4827e-02, -5.6394e-02,  8.5323e-03,
         1.9419e-02,  1.9413e-02, -8.2380e-03, -3.1306e-02, -1.0680e-02,
        -6.2193e-03, -2.7547e-04, -2.6264e-02, -2.0376e-02, -9.5900e-03,
        -1.3520e-02,  3.9128e-02, -3.3298e-03]

In [12]:
from scipy.spatial.distance import cosine
cosine(list1, list2)

5.981987007963596e-07